# Review Disease Ontology Mappings to ImmPort

The information used in this notebook as produced by using the NCBO Annotator to map ImmPort study information to Disease Ontology terms. The following fields from the Study data were submitted to NCBO Annotator: 

1. brief_title
2. condition_studied
3. description
4. endpoints
5. hypothesis

The results from the NCBO Annotator API were parsed, and written to the annotation.txt file. The output columns are:

1. Study_accession
2. Field - this identifies which field from ImmPort was intergated
3. DOID - Disease Ontology Identifier
4. Name - DO name
5. Match - word or phrase that matched DO
6. Match Type - PREF or SYN

In [1]:
import pandas as pd
import qgrid
import json
import re

from IPython.display import display, HTML
import ipywidgets as widgets

# Display Annotation Information

In [2]:
annotation = pd.read_csv("./data/annotation.txt", sep="\t")
annotation_widget = qgrid.show_grid(annotation, show_toolbar=False)
annotation_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [3]:
annotation_widget = qgrid.show_grid(annotation, show_toolbar=False)

In [4]:
annotation_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Review Individual Studies

In [5]:
def sortSDY(s):
    return int(s.replace("SDY", ""))

studies = sorted(annotation['Study_accession'].unique(),key=sortSDY)

In [6]:
study_dropdown = widgets.Dropdown(
    options=studies,
    value = studies[0],
    description = "Study: ",
    layout={'height': '50px', 'width': '450px'})

In [7]:
def filter_data(study_accession):
    data = annotation[annotation['Study_accession']==study_accession]
    return data.drop_duplicates()

In [8]:
def display_filtered_results(data):
    filtered_widget = qgrid.show_grid(data, show_toolbar=False)
    return filtered_widget

In [9]:
def load_study(study_accession):
    file_name = "./data/" + study_accession + ".json"
    study = json.load(open(file_name, "r"))
    return study

In [10]:
def replace_all(pattern, string):
    occurences = re.findall(pattern, string, re.IGNORECASE)
    for occurence in occurences:
         string = string.replace(occurence, "<span style='color: red'>" + pattern + "</span>")
        
    return string


def prepare_study(study, matches):
    for match in matches:
        study['brief_title'] = replace_all(match, study['brief_title'])
        study['condition_studied'] = replace_all(match, study['condition_studied'])
        study['description'] = replace_all(match, study['description'])
        study['hypothesis'] = replace_all(match, study['hypothesis'])
        study['endpoints'] = replace_all(match, study['endpoints'])
                               
    rows = []
    rows.append("<h3>Study: " + study['study_accession'])
    rows.append("<br>")
    rows.append("<table style='border: 1px solid black; font-size: 14px'>")
    rows.append("<tr>")
    rows.append("<td style='valign: top; border: 1px solid black'>brief_title</td>")
    rows.append("<td style='valign: top; border: 1px solid black'>" + study['brief_title'] + "</td>")
    rows.append("</tr>")
    rows.append("<tr>")
    rows.append("<td style='valign: top; border: 1px solid black'>condition_studied</td>")
    rows.append("<td style='valign: top; border: 1px solid black'>" + study['condition_studied'] + "</td>")
    rows.append("</tr>")
    rows.append("<tr>")
    rows.append("<td style='valign: top; border: 1px solid black'>description</td>")
    rows.append("<td style='valign: top; border: 1px solid black'>" + study['description'] + "</td>")
    rows.append("</tr>")
    rows.append("<tr>")
    rows.append("<td style='valign: top; border: 1px solid black'>hypothesis</td>")
    rows.append("<td style='valign: top; border: 1px solid black'>" + study['hypothesis'] + "</td>")
    rows.append("</tr>")
    rows.append("<tr>")
    rows.append("<td style='valign: top; border: 1px solid black'>endpoints</td>")
    rows.append("<td style='valign: top; border: 1px solid black'>" + study['endpoints'] + "</td>")
    rows.append("</tr>")
    return "\n".join(rows)

# Constuct UI

In [11]:
row1 = widgets.HBox([study_dropdown])
out1 = widgets.Output(layout={'height': 'auto', 'width': 'auto'})
#out2 = widgets.Output(layout={'height': 'auto', 'width': '800px'})
out2 = widgets.HTML(layout={'height': 'auto', 'width': 'auto'})
ui = widgets.VBox([row1, out1, out2], 
          layout={ 'height': 'auto', 'border': '2px solid blue', 'margin': '2px 2px 2px 2px'})

In [12]:
def display_ui(event):
    study_accession = study_dropdown.value
    #out2.value=""
    data = filter_data(study_accession)
    matches = data['Match'].unique()
    #fw = display_filtered_results(data)
    study = load_study(study_accession)
    out1.clear_output()
    #out2.value=""
    with out1:
        display(data)
    out2.value=prepare_study(study, matches)
    #out2.clear_output()
    #with out2:
        #display(prepare_study(study))

In [13]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [14]:
study_dropdown.observe(display_ui)
display_ui(None)
ui